In [65]:
a = b'\xbf\xbd'

In [67]:
a.decode('utf8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 0: invalid start byte

In [70]:
b = '\u4f60\u60f3\u8981\u8bbf\u95ee\u7684\u8bb0\u5f55\u6ca1\u6709\u627e\u5230'
print(b)

你想要访问的记录没有找到


In [45]:
import requests
import re

In [21]:
def getLatestChgFromFt(sCode):
    # SPGOGUP:REU
    resp = requests.get('https://markets.ft.com/data/indices/tearsheet/summary?s={}'.format(sCode))

    m = re.search("today's change", resp.text, re.IGNORECASE)

    iBgn = m.start()

    rexPct = re.compile("([-\.0-9]+?)%")

    m = rexPct.search(resp.text, iBgn)

    fPct = float(m.group(1))
    
    return fPct


In [22]:
print(getLatestChgFromFt('SPGOGUP:REU'))

-0.51


In [65]:
def getChgFromFt(sCode):
    # Code example: 'SPGOGUP:REU', 'SPGNRET:REU'
    resp = requests.get('https://markets.ft.com/data/indices/tearsheet/historical?s={}'.format(sCode))

    m = re.search("table class.*?historical-prices", resp.text, re.IGNORECASE)

    iBgn = resp.text.find('<tbody>', m.start())
    iEnd = resp.text.find('</tbody>', iBgn)

    rexRow = re.compile("<tr>(.*?)</tr>", re.DOTALL)
    rexCol = re.compile("<td.*?>([^<>]+?)<.*?/td>", re.DOTALL)

    lsRows = rexRow.findall(resp.text[iBgn:iEnd])

    lsChg = []
    i = 0
    for sRow in lsRows:
        lsCols = rexCol.findall(sRow)   # Date, Open, High, Low, Close, Volume
        dtDate = datetime.strptime(lsCols[0], '%A, %B %d, %Y')    # 'Friday, March 10, 2017'
        sDate = dtDate.strftime('%Y%m%d')
        fClose = float(lsCols[4].replace(',', ''))
        lsChg.append([sDate, fClose])
        if i > 0:
            fNxtCls = lsChg[i-1][1]
            fChg = (fNxtCls/fClose - 1) * 100
            lsChg[i-1].append(fChg)
        i += 1
    
    return (lsChg[0][0], lsChg[0][2])
#end def

In [66]:
print(getChgFromFt('SPGOGUP:REU'))

('20170310', 0.523560209424101)


In [92]:
from datetime import datetime, timezone, timedelta

In [44]:
def getIdxChgFromYahoo(sCode, sDftDate = None):
    # Example: getIdxChgFromYahoo('^SPGNRUP')
    from yahoo_finance import Share
    from datetime import datetime
    from datetime import timezone

    idx = Share(sCode)

    fPct = float(idx.get_percent_change().replace('%', ''))
    
    sDate = sDftDate
    if not sDftDate:
        dt = datetime.strptime(idx.get_trade_datetime(), '%Y-%m-%d %H:%M:%S %Z%z')   # e.g. '2017-03-13 02:31:00 UTC+0000'
        dt = dt.astimezone(tz=None)
        sDate = dt.strftime('%Y%m%d')
        
    return (sDate, fPct)

In [83]:
dt.tzinfo

datetime.timezone(datetime.timedelta(0), 'UTC')

In [43]:
print(getIdxChgFromYahoo('^SPGNRUP'))

('2017-03-13', 0.64)


In [96]:
def getChgFromGoogle(sCode, sDftDate = None):
    import json
    import dateutil.parser as parser

    # 12-day history API:
    # http://www.networkerror.org/component/content/44.html?task=view
    # Code example: 'INDEXSP:SPGNRUP' or just 'SPGNRUP'
    #sUrl = 'https://www.google.com/finance/info?infotype=infoquoteall&q=INDEXSP:SPGNRUP'
    sUrl = 'https://www.google.com/finance/info?q={}'.format(sCode)
    resp = requests.get(sUrl)

    ls = json.loads(resp.text[3:])
    d = ls[0]
    fPct = float(d['cp'])
    #print(fPct)
    
    sDate = sDftDate
    if not sDftDate:
        sTime = d['lt_dts'][:10] + ' ' + d['ltt']
        #print(sTime)
        # '2017-03-13 12:29AM EDT'
        dt = parser.parse(sTime)
        #print(dt)
        # Add US timezone info, and translate to local time
        dt = dt.replace(tzinfo=timezone(timedelta(hours=-4))).astimezone()
        sDate = dt.strftime('%Y%m%d')
    
    return (sDate, fPct)

In [97]:
print(getChgFromGoogle('SPGNRUP'))

('20170313', 0.73)


In [98]:
x = eval("getChgFromGoogle('SPGNRUP')")
print(x)

('20170313', 0.72)


In [91]:
import pytz

In [100]:
parser.parse('2017-03-13 12:29AM EDT')

datetime.datetime(2017, 3, 13, 0, 29)

In [25]:
dt = datetime.strptime(sp.get_trade_datetime(), '%Y-%m-%d %H:%M:%S %Z%z')

In [26]:
dt = dt.replace(tzinfo=timezone.utc)

In [27]:
dt.astimezone(tz=None)

datetime.datetime(2017, 3, 13, 10, 11, tzinfo=datetime.timezone(datetime.timedelta(0, 28800), 'ÖÐ¹ú±ê×¼Ê±¼ä'))

In [29]:
dt.astimezone(tz=None).strftime('%Y-%m-%d %H:%M:%S')

'2017-03-13 10:11:00'

In [36]:
aapl = Share('AAPL')

In [37]:
aapl.get_trade_datetime()

'2017-03-10 22:00:00 UTC+0000'

In [ ]:
r = ar[1]

In [ ]:
np.sum(w*r)/np.sum(w)

In [ ]:
totw = 0.0
totr = 0.0
for i in range(0, len(r)):
    if r[i] == 0:
        modr = totr / totw
    else:
        modr = r[i]
    totr += ar[0, i] * modr
    totw += ar[0, i]
print(totr/np.sum(ar[0]))

In [ ]:
totw = 0.0
wr = 0.0
for i in range(0, len(r)):
    if r[i] != 0:
        wr = (wr * totw + r[i] * ar[0, i]) / (totw + ar[0, i])
    totw += ar[0, i]
print(wr)

In [ ]:
c1 = np.random.rand(10)
c2 = np.random.rand(10)
x1 = np.random.randint(0, 9, size=4)
x2 = np.random.randint(0, 9, size=4)
c1[x1] = 0.0
c2[x2] = 0.0

In [ ]:
c1

In [ ]:
c2

In [ ]:
w1 = 0.3
w2 = 0.5
(w1 * c1 + w2 *c2) / (w1 + w2)

In [ ]:
f1 = (c1!=0)
f2 = (c2!=0)

In [ ]:
f1

In [ ]:
(w1 * c1 + w2 *c2) / (f1*w1 + f2*w2)

In [ ]:
f1*w1 + f2*w2

In [ ]:
w1 * c1 + w2 *c2

In [ ]:
w1 * c1 *f1 + w2 * c2*f2

In [ ]:
ar_w = f1*w1 + f2*w2
ar_w[ar_w == 0] = 1

In [ ]:
ar_w

In [ ]:
(w1 * c1 + w2 *c2) / ar_w

In [ ]:
os.chdir('../Gold/')

In [ ]:
dfStk = pd.DataFrame.from_csv("data/{}_rtn.csv".format("600547"), encoding = 'gbk')

In [ ]:
srWght = 0.4951 * ((dfStk.TRADE_STATUS == '交易') | (dfStk.PCT_CHG != 0))

In [ ]:
srWghtSum = pd.Series()

In [ ]:
srWghtSum += 0.4951 * ((dfStk.TRADE_STATUS == '交易') | (dfStk.PCT_CHG != 0))

In [ ]:
sum(srWght == 0)

In [ ]:
srA = dfStk.PCT_CHG * srWght
srA.name = 'VirtRtn'
srB = dfStk.PCT_CHG * 0.4951
srB.name = 'PtflRtn'

In [ ]:
dfAB = pd.concat([srA, srB], axis = 1)

In [ ]:
dfAB.VirtRtn.sum()

In [ ]:
dfAB.PtflRtn.sum()

In [ ]:
dfAB.VirtRtn.name

In [ ]:
ts = dfStk.index[3]

In [ ]:
dfStk.loc[ts]

In [ ]:
dfStk.loc[ts, 'CLOSE']

In [ ]:
dfStk.index = dfStk.index.normalize()

In [ ]:
dfStk.index

In [ ]:
round(255, -2)

In [ ]:
"{:.2%}".format(3.14)

In [ ]:
sr=pd.Series({'d':1.2, 'sl':3.4, 'ri':np.nan})

In [ ]:
sr[~sr.isnull()]

In [ ]:
sr

In [ ]:
fill_between_demo.py

In [ ]:
os.getcwd()

In [ ]:
os.chdir('../IndustryFlow')

In [ ]:
plt.bar([1,2,3],[1,2,3], color = 'orangered', edgecolor = 'none')

In [ ]:
plt.show()

In [ ]:
import requests

In [ ]:
r = requests.get('https://beta.cebupacificair.com/Flight/InternalSelect?s=true&o1=PVG&d1=MPH&dd1=2017-05-24&dd2=2017-05-28&r=true&mon=true', verify=False)

In [ ]:
headers = {'Referer':'https://beta.cebupacificair.com/Flight/Select'}

In [ ]:
f = open("x.html", 'w')

In [ ]:
f.write(r.text)

In [ ]:
f.close()

In [ ]:
dGoldStk = {'002237.SZ': 0.24604859603625656, '600655.SH': 0.29655154244190385, '600362.SH': 
 0.20333606415106328, '002155.SZ': 0.40589162439651888, '601899.SH': 0.20039110916621541, 
 '600489.SH': 0.65383478245376014, '600531.SH': 0.24957306413717745, '600547.SH': 
 0.63773575732003318}

In [ ]:
def genGoldConceptRtn(dGoldStk = None, dtEnd = None, nSample = None, dStkDf = None):
    os.chdir('../Gold/')
    if dGoldStk is None:
        #dGoldStk = {"600547": 0.458495072, "600489": 0.402015087, "002155": 0.391238013, "601069": 0.361354263, "600988": 0.359376271, "002716": 0.336558704, "002237": 0.329655207}
        dGoldStk = {"600547":0.495101653, "600489":0.4703083, "002155":0.384113351, "601069":0.359351555, "002237":0.352791002, "002716":0.333648661, "600531":0.30050461, "601899":0.289093361}
    #dfConcept = pd.DataFrame()
    srConcept = pd.Series()
    srWghtSum = pd.Series()
    for sStkCode, fWght in dGoldStk.items():
        sDigCode = sStkCode[:6]     # six-digit code
        if dStkDf is None:
            dfStk = pd.DataFrame.from_csv("data/{}_rtn.csv".format(sDigCode), encoding = 'gbk')
        else:
            dfStk = dStkDf[sStkCode]
        srWeight = dGoldStk[sStkCode] * ((dfStk.TRADE_STATUS == '交易') | (dfStk.PCT_CHG != 0))
        #print(sStkCode, min(dfStk.index))
        if dtEnd is None:
            srPctRtn = dfStk.PCT_CHG
        else:
            srPctRtn = dfStk.PCT_CHG.loc[:dtEnd].iloc[-nSample:]
            srWeight = srWeight.loc[:dtEnd].iloc[-nSample:]
        srStkRtn = np.log(srPctRtn/100 + 1) * 100       # translate to log-return
        # Notice if the fill_value is not specified, an empty Series + another Series will be filled with NaN
        srConcept = srConcept.add(srStkRtn * fWght, fill_value = 0)     # the return is 0 when stock is suspended
        #dfConcept = dfConcept.add(srStkRtn.to_frame() * fWght, fill_value = 0)
        # Suspended return (=0) does not count weight, to prevent distortions in regression results.
        srWghtSum = srWghtSum.add(srWeight, fill_value = 0)
    #end for
    srPtflRtn = srConcept / sum(dGoldStk.values())      # actual return of the portfolio
    srPtflRtn.name = 'PtflRtn'
    srVirtRtn = srConcept / srWghtSum                   # return series for regression computation
    srVirtRtn.name = 'VirtRtn'
    #dfConcept = dfConcept / sum(dGoldStk.values())
    #dfConcept = srConcept.to_frame()
    dfConcept = pd.concat([srPtflRtn, srVirtRtn], axis = 1)
    return dfConcept
#end def

In [ ]:
dfConcept = genGoldConceptRtn(dGoldStk)

In [ ]:
dfConcept

In [ ]:
sFolder = '../Gold/'
os.chdir(sFolder)

In [ ]:
sGldRtnFile = 'au_sett_rtn.csv'
dfGld = pd.DataFrame.from_csv(sGldRtnFile)

srGldRtn = dfGld.loc[:,'AdjRtn']    # Adjusted returns which include rolling costs
srGldRtn.name = 'AU'

In [ ]:
dParams = {'CSI500': 0.87164886908927519, 'Intercept': -0.10821257718169057, 'AU': 0.78353383345943362}

In [ ]:
sIdxCode = 'CSI500'
sIdxFile = 'data/{}_rtn.csv'.format('000905.SH')
dfIdxRtn = pd.DataFrame.from_csv(sIdxFile)
dfIdxRtn.LOG_CHG = np.log(dfIdxRtn.PCT_CHG/100+1)*100       # convert to log return

In [ ]:
srStkPfRtn = dfConcept.PtflRtn
srIdxRtn = dfIdxRtn.LOG_CHG
srIdxRtn.name = sIdxCode
dfCmp = pd.concat([srGldRtn, srStkPfRtn, srIdxRtn], axis = 1, join = 'inner')

In [ ]:
dfCmp

In [ ]:
srTotRtn = srStkPfRtn - dParams['AU']*srGldRtn - dParams['CSI500']*srIdxRtn

In [ ]:
plt.plot(srTotRtn[250:].index, srTotRtn[250:].cumsum())
plt.show()

In [ ]:
os.environ['PATH']

In [ ]:
import sys
sys.version_info()

In [ ]:
sys.version_info

In [ ]:
import numpy
#import sys
#sys.path.append('D:\Python35\Lib\site-packages\cvxopt-0+unknown-py3.5-win-amd64.egg\cvxopt')
import cvxopt

In [ ]:
from PyQt4.QtCore import QTimer

In [ ]:
timer = QTimer()

In [ ]:
from simplecrypt import encrypt, decrypt

In [ ]:
encrypt('hello world', 'which one is the real data')

In [ ]:
decrypt('hello worl', b'sc\x00\x02\xc27\xca\xb7\x90\xdf?\x19\x82\x13\xda\x82\x1a!\xd4\xf3\xda<1\xbc*\xc9>yJ$^\xd2\n\xdf\x04\x0b\x89\xcaR\x06\xab\xd9\xf8\xbex\x9b\xec\x85\xb1\xbf\xbc\xcfB\xcf:\x7f\x91\x95|\x15x\xb9\x1b\xb9w\x83\xe6\x86Z\xcb\x171\x9a\x82D>\x1bOO)\\+\xb0~C\xdc\x0b\xa1l\xc6\x163\xd3"')

In [ ]:
from collections import defaultdict

In [ ]:
from WindPy import *
w.start()

In [ ]:
dBook = defaultdict(dict)

In [ ]:
def printOrderBook(dBook):
    for sCode, dDepth in dBook.items():
        print('{}: {}'.format(sCode, dDepth))

In [ ]:
def myCallback(rcvData):
    #print("")
    #print(rcvData.Times)
    #print(rcvData.Codes)
    #print(rcvData.Fields)
    #print(rcvData.Data)
    for i in range(0, len(rcvData.Codes)):
        sCode = rcvData.Codes[i]
        dDepth = dBook[sCode]
        for j in range(0, len(rcvData.Fields)):
            sField = rcvData.Fields[j]
            dDepth[sField] = rcvData.Data[j][i]


In [ ]:
res = w.wsq("160105.SZ,160106.SZ,160119.SZ,160125.SZ,160130.SZ,160131.SZ,160133.SZ,160211.SZ,160212.SZ,160215.SZ,160216.SZ,160220.SZ,160223.SZ,160225.SZ,160311.SZ,160314.SZ,160322.SZ,160323.SZ,160415.SZ,160416.SZ,160422.SZ,160505.SZ,160512.SZ,160513.SZ,160515.SZ,160607.SZ,160610.SZ,160611.SZ,160613.SZ,160615.SZ,160616.SZ,160617.SZ,160618.SZ,160621.SZ,160622.SZ,160635.SZ,160706.SZ,160716.SZ,160717.SZ,160719.SZ,160720.SZ,160805.SZ,160806.SZ,160807.SZ,160810.SZ,160812.SZ,160813.SZ,160910.SZ,160915.SZ,160916.SZ,160918.SZ,160919.SZ,160922.SZ,160923.SZ,161005.SZ,161010.SZ,161015.SZ,161017.SZ,161033.SZ,161035.SZ,161036.SZ,161115.SZ,161116.SZ,161117.SZ,161119.SZ,161124.SZ,161125.SZ,161126.SZ,161127.SZ,161128.SZ,161129.SZ,161210.SZ,161213.SZ,161216.SZ,161217.SZ,161219.SZ,161222.SZ,161224.SZ,161225.SZ,161226.SZ,161227.SZ,161229.SZ,161505.SZ,161506.SZ,161607.SZ,161610.SZ,161614.SZ,161626.SZ,161630.SZ,161631.SZ,161706.SZ,161713.SZ,161714.SZ,161716.SZ,161722.SZ,161810.SZ,161813.SZ,161815.SZ,161820.SZ,161821.SZ,161823.SZ,161903.SZ,161907.SZ,161908.SZ,162006.SZ,162105.SZ,162107.SZ,162108.SZ,162207.SZ,162215.SZ,162307.SZ,162308.SZ,162411.SZ,162414.SZ,162415.SZ,162510.SZ,162511.SZ,162605.SZ,162607.SZ,162703.SZ,162711.SZ,162712.SZ,162715.SZ,162719.SZ,163001.SZ,163003.SZ,163005.SZ,163110.SZ,163119.SZ,163208.SZ,163302.SZ,163402.SZ,163407.SZ,163409.SZ,163412.SZ,163415.SZ,163503.SZ,163801.SZ,163819.SZ,163821.SZ,163824.SZ,163907.SZ,164105.SZ,164206.SZ,164208.SZ,164210.SZ,164302.SZ,164403.SZ,164606.SZ,164701.SZ,164703.SZ,164808.SZ,164812.SZ,164814.SZ,164815.SZ,164902.SZ,164906.SZ,164908.SZ,165309.SZ,165311.SZ,165313.SZ,165508.SZ,165509.SZ,165510.SZ,165512.SZ,165513.SZ,165516.SZ,165517.SZ,165525.SZ,165526.SZ,165705.SZ,165806.SZ,165807.SZ,166001.SZ,166006.SZ,166007.SZ,166008.SZ,166009.SZ,166011.SZ,166012.SZ,166016.SZ,166105.SZ,166401.SZ,167501.SZ,167702.SZ,501000.SH,501001.SH,501002.SH,501005.SH,501007.SH,501008.SH,501009.SH,501010.SH,501011.SH,501012.SH,501018.SH,501021.SH,501023.SH,501025.SH,501029.SH,501030.SH,501031.SH,501050.SH,501100.SH,501300.SH,502000.SH,502003.SH,502006.SH,502010.SH,502013.SH,502016.SH,502020.SH,502023.SH,502026.SH,502036.SH,502040.SH,502048.SH,502053.SH,502056.SH", "rt_latest,rt_ask1,rt_bid1,rt_asize1,rt_bsize1,rt_pct_chg,rt_pre_close,rt_ask2,rt_bid2", func=myCallback)

In [ ]:
printOrderBook(dBook)

In [ ]:
print(dBook['162411.SZ'])

In [ ]:
w.cancelRequest(res.RequestID)